<div
  style="
    background-color: #f0f0f0;
    color:rgb(56, 56, 56);
    padding: 8px;
    display: flex;
    align-items: center;
    gap: 100px;
  "
>
  <img src="./images/brand.svg" style="max-height: 80px;">
  <strong>
    AI Saga: Data Science and Machine Learning</br>
    2.lab.1. Wisconsin Cancer Classification
  </strong>
</div>

### Background

You will continue working with the Wisconsin Diagnostic Cancer Dataset, but this time implementing classification models using scikit-learn.

The dataset includes:
- 569 instances
- 30 numeric features computed from the cell nuclei present in the image
- Binary classification: **Malignant** or **Benign** diagnosis

### Description

Your task is to implement logistic regression using scikit-learn to:

- Implement logistic regression using all features
- Find the best performing pair of features for classification
- Compare the accuracy between using all features vs. the best pair
- Visualize the decision boundary for the best feature pair

### Deliverables

- A **ipynb** (Jupyter Notebook) file called **wisconsin-cancer-classification.ipynb**
- The notebook should include visualizations of the decision boundaries
- A clear comparison of performance between using all features vs. the best pair
- Use the provided template as a starting point

# Clasificación de Cáncer con Scikit-Learn

>nota: iniciare nuevamente las librerias para no arruinar o cambiar mucho la impelmentacion anteriror

Cargar dataset de scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from rich.table import Table
from rich.console import Console
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from itertools import combinations


cancer_data = load_breast_cancer()
X = cancer_data.data
y = cancer_data.target
feature_names = cancer_data.feature_names
console = Console()

## reprocesamiento con Scikit-Learn


aqui pues explico lo que entendi del codigo pues me base muchoe en el propuesto en clase

- train_test_split() divide automáticamente los datos aleatoriamente
-  StandardScaler() calcula la media y desviación estándar del entrenamiento con fit_transform() y luego aplica la misma transformación al test con transform().



aqui basicamente lo qeu hhacemos es La normalización  porque las mediciones en este caso tienen escalas muy diferentes en todas las varibels o coliumnas tipo el radio promedio puede ser 14.0 mientras que la dimensión  es 0.06 entonces Sin esta estandarización, las características con valores más grandes dominarían por asi decir  la clasificación.



In [ ]:
# división de datos con scikit-learn
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# Normalización con scikit-learn
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Regresión Logística con Todas las Características


- LogisticRegression() crea el modelo
- fit() entrena con los datos normalizados, 
- predict() genera las clasificaciones y accuracy_score() calcula el porcentaje de todos los  aciertos.

aqui pues loq eu ahcemos es considerar todas  las medicioones para  determinar los tumores osea sis es maligno o benigno, cada caracteristica va a recibir un peso para indicar su importancia en la decision del final 

In [ ]:
model_all_features = LogisticRegression(random_state=42, max_iter=1000)
model_all_features.fit(X_train_scaled, y_train)

y_pred_all = model_all_features.predict(X_test_scaled)
accuracy_all = accuracy_score(y_test, y_pred_all)

## Busqueda del Mejor Par de Características

- combinations() genera todas las parejas posibles. 
- El bucle entrena en un LogisticRegression() independiente para cada par usando indexación de arrays (X[:, [idx1, idx2]]) para seleccionar solo 2 columnas.


aqui estos metodos basicamente lo qeu vamos hacer es evlauar 435 combinaciones posibles con los pares de caracteristicas para de esa manera saber cuales son las dos mediciones mas efectivas para distingir entre los tumores, en este caso son worst radius/ worst fractal dimension


In [ ]:
# busqueda  usando scikit-learn
best_accuracy = 0
best_pair = None
best_indices = None

for idx1, idx2 in combinations(range(X.shape[1]), 2):
    X_train_pair = X_train_scaled[:, [idx1, idx2]]
    X_test_pair = X_test_scaled[:, [idx1, idx2]]

    model_pair = LogisticRegression(random_state=42, max_iter=1000)
    model_pair.fit(X_train_pair, y_train)

    y_pred_pair = model_pair.predict(X_test_pair)
    accuracy_pair = accuracy_score(y_test, y_pred_pair)

    if accuracy_pair > best_accuracy:
        best_accuracy = accuracy_pair
        best_pair = (feature_names[idx1], feature_names[idx2])
        best_indices = (idx1, idx2)

# aqui se entrena al  modelo final con mejor par
X_train_best = X_train_scaled[:, list(best_indices)]
X_test_best = X_test_scaled[:, list(best_indices)]
model_best_pair = LogisticRegression(random_state=42, max_iter=1000)
model_best_pair.fit(X_train_best, y_train)

print(f"Mejor par encontrado: {best_pair}")
print(f"Precisión del mejor par: {best_accuracy:.4f}")

##  Visualizacion de la Frontera de Decision


- plt.scatter() con indexación booleana (y_train == 0) separa los puntos por color. 
- np.meshgrid() crea una rejilla de puntos, 
- model.predict() clasifica cada punto de la rejilla y plt.contour() dibuja la línea donde va a cambiar  la predicción.

- La gráfica muestra una clara  separación  entre los puntos rojos y azules. Los puntos rojos (tumores malignos) tienden a concentrarse en la región superior derech lo que nos indica que los tumores malignos tienen valores más altos tanto en "worst radius" como en "worst fractal dimension"  mientra que 

- Los puntos azules, qlos tumores benignos, se agrupan en la zona inferior izquierda indicando que tienen células más pequeñas y de una forma regular. La línea verde en este caso va  a ser el límite que divide los resultados todo lo que queda arriba y a la derecha de ella se clasifica como tumor maligno, mientras que lo que está abajo y a la izquierda se consideraun tumor  benigno

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(
    X_train_best[y_train == 0, 0],
    X_train_best[y_train == 0, 1],
    color="red",
    label="Maligno",
    alpha=0.6,
)
plt.scatter(
    X_train_best[y_train == 1, 0],
    X_train_best[y_train == 1, 1],
    color="blue",
    label="Benigno",
    alpha=0.6,
)

# Creacion frontera de decisión
h = 0.02
x_min, x_max = X_train_best[:, 0].min() - 1, X_train_best[:, 0].max() + 1
y_min, y_max = X_train_best[:, 1].min() - 1, X_train_best[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

Z = model_best_pair.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contour(xx, yy, Z, levels=[0.5], colors="green", linewidths=2, linestyles="--")
plt.xlabel(best_pair[0])
plt.ylabel(best_pair[1])
plt.title(f"Frontera de Decisión - {best_pair[0]} vs {best_pair[1]}")
plt.legend()
plt.show()

## Comparación de Modelos


- rich.Table() crea una tabla formateada para mostrar los resultados de manera clara, comparando las precisiones obtenidas con accuracy_score().

al anlizar la tabla podemos decir que: 

- el modelo que usa solo 2 características logra mayor precisión (98.25%) que el modelo con 30 características (97.37%) lo ques sugiere que las mediciones "worst radius" y "worst fractal dimension" si tienen la  información más crítica para distinguir o definir  entre tumores malignos y benignos.

In [ ]:
comparison_table = Table(
    title="Comparación de Modelos con Scikit-Learn",
    show_header=True,
    header_style="bold",
)

comparison_table.add_column("Modelo", justify="left", style="cyan")
comparison_table.add_column("Características", justify="center", style="yellow")
comparison_table.add_column("Precisión", justify="right", style="green")

comparison_table.add_row(
    "Todas las características", f"{X.shape[1]} características", f"{accuracy_all:.4f}"
)

comparison_table.add_row(
    "Mejor par de características",
    f"{best_pair[0][:20]}...\n{best_pair[1][:20]}...",
    f"{best_accuracy:.4f}",
)

console.print(comparison_table)

## Anlisis final

usar solo 2 características del tumor (tamaño e irregularidad de la forma) funcionó mejor que usar las 30 disponibles pues usando  regresión logística.graficamos los resultados, vimos una línea muy clara que separa los tumores malignos (puntos rojos) de los benignos (puntos azules), confirmando que estas dos medidas fueron las mas criticas en  los patrones reales de la enfermedad. Las implicaciones son muy prácticas: análisis más rápidos,fáciles de interpretar en caso de  uso en la medicina , y el uso de la libreria simplifico muchisimo mas el proceso la regresioin  y aplicacion mateematica .en resumen si se obtiene caracteristicas escneciales se puede con ese enfoque tener un mejor anaslisis para sitribucion de datos.